In [1]:
%load_ext advent_of_code

In [2]:
import re
import math
import itertools

from collections import defaultdict, Counter

from tqdm.auto import tqdm

# Day 1

In [3]:
%%aoc 1 1

answer = 0
for line in inputs.strip().split("\n"):
    num = [c for c in line if c.isdigit()]
    answer += int(num[0] + num[-1])

Execution time: 0.0027 seconds
Correct!


In [4]:
%%aoc 1 2

digits = ["one", "two", "three", "four", "five", "six", "seven", "eight", "nine"]
digit_map = {d: i for i, d in enumerate(digits, 1)}

answer = 0
for line in inputs.strip().split("\n"):
    num = re.findall("(?=(\d|"+"|".join(digits)+"))", line)
    num = [digit_map.get(i, None) or int(i) for i in num]
    answer += num[0] * 10 + num[-1]

Execution time: 0.0062 seconds
Correct!


# Day 2

In [5]:
%%aoc 2 1

total = {"red": 12, "green": 13, "blue": 14}

answer = 0

for game in inputs.strip().split("\n"):
    num, rounds = game.split(": ", 1)
    num = int(num[5:])

    possible = True
    for round in rounds.split("; "):
        for cubes in round.split(", "):
            count, color = cubes.split()
            if int(count) > total[color]:
                possible = False

    if possible:
        answer += num

Execution time: 0.0022 seconds
Correct!


In [6]:
%%aoc 2 2

answer = 0

for game in inputs.strip().split("\n"):
    num, rounds = game.split(": ", 1)
    num = int(num[5:])

    total = {"red": 0, "green": 0, "blue": 0}


    for round in rounds.split("; "):
        for cubes in round.split(", "):
            count, color = cubes.split()
            total[color] = max(total[color], int(count))

    power = 1
    for count in total.values():
        power *= count

    answer += power

Execution time: 0.0027 seconds
Correct!


# Day 3

In [7]:
%%aoc 3 1

grid = inputs.strip().split("\n")

grid = ["." + line + "." for line in grid]
grid = ["."*len(grid[0])] + grid + ["."*len(grid[0])]

answer = 0
for i, line in enumerate(grid[1:-1], 1):
    for num in re.finditer(r"(?<!\d)\d+(?=.)", line):
        start, end = num.span()
        if not (set(grid[i-1][start-1:end+1] + grid[i+1][start-1:end+1] + line[start-1] + line[end]) == {"."}):
            answer += int(num.group())

Execution time: 0.0051 seconds
Correct!


In [8]:
%%aoc 3 2

grid = inputs.strip().split("\n")

grid = ["." + line + "." for line in grid]
grid = ["."*len(grid[0])] + grid + ["."*len(grid[0])]

answer = 0

gear_values = defaultdict(list)

for i, line in enumerate(grid[1:-1], 1):
    for num in re.finditer(r"(?<!\d)\d+(?=.)", line):
        start, end = num.span()
        for row_i in range(i-1, i+2):
            row = grid[row_i][start-1:end+1]
            for gear in re.finditer(r"\*", row):
                gear_values[(row_i, start+gear.start())].append(int(num.group()))

for nums in gear_values.values():
    if len(nums) == 2:
        answer += nums[0] * nums[1]

Execution time: 0.0092 seconds
Correct!


# Day 4

In [9]:
inputs = """Card 1: 41 48 83 86 17 | 83 86  6 31 17  9 48 53
Card 2: 13 32 20 16 61 | 61 30 68 82 17 32 24 19
Card 3:  1 21 53 59 44 | 69 82 63 72 16 21 14  1
Card 4: 41 92 73 84 69 | 59 84 76 51 58  5 54 83
Card 5: 87 83 26 28 32 | 88 30 70 12 93 22 82 36
Card 6: 31 18 13 56 72 | 74 77 10 23 35 67 36 11
"""

In [10]:
%%aoc 4 1

answer = 0

for card in inputs.strip().split("\n"):
    i, win, nums = re.match(r"Card +(\d+): ([0-9 ]+) \| ([0-9 ]+)", card).groups()
    win = Counter(win.split())
    nums = Counter(nums.split())

    answer += int(2**((win & nums).total()-1))

Execution time: 0.0042 seconds
Correct!


In [11]:
%%aoc 4 2

muls = [1 for card in inputs.strip().split("\n")]

for i, card in enumerate(inputs.strip().split("\n")):
    win, nums = re.match(r"Card +\d+: ([0-9 ]+) \| ([0-9 ]+)", card).groups()
    win = Counter(win.split())
    nums = Counter(nums.split())

    for j in range(i+1, i+1+(win & nums).total()):
        muls[j] += muls[i]

answer = sum(muls)

Execution time: 0.0048 seconds
Correct!


# Day 5

In [12]:
%%aoc 5 1

seeds, *maps = inputs.strip().split("\n\n")

seeds = [int(seed) for seed in seeds.split()[1:]]
maps = [[[int(x) for x in part.split()] for part in layer.split("\n")[1:]] for layer in maps]

answer = float("inf")
for seed in seeds:
    for layer in maps:
        for dest, source, n in layer:
            if source <= seed < source + n:
                seed = seed - source + dest
                break
    answer = min(answer, seed)

Execution time: 0.0022 seconds
Correct!


In [13]:
%%aoc 5 2

seeds, *maps = inputs.strip().split("\n\n")

seeds = [int(seed) for seed in seeds.split()[1:]]
seeds = [[(s, s+n)] for s, n in zip(seeds[::2], seeds[1::2])]

maps = [[[int(x) for x in part.split()] for part in layer.split("\n")[1:]] for layer in maps]
maps = [[(dest_start, source_start, source_start + layer_n) for dest_start, source_start, layer_n in layer] for layer in maps]

answer = float("inf")
for seed in seeds:
    for layer in maps:
        new_seed = []
        while seed:
            reg_start, reg_end = seed.pop()
            for dest_start, source_start, source_end in layer:
                overlap = (max(reg_start, source_start), min(reg_end, source_end))
                if overlap[0] < overlap[1]:                    
                    new_seed.append((overlap[0] - source_start + dest_start, overlap[1] - source_start + dest_start))
                    if reg_start < source_start:
                        seed.append((reg_start, source_start))
                    if reg_end > source_end:
                        seed.append((source_end, reg_end))
                    break
            else:
                new_seed.append((reg_start, reg_end))
    
        seed = new_seed
    answer = min(answer, *(reg[0] for reg in seed))

Execution time: 0.0083 seconds
Correct!


# Day 6

In [14]:
%%aoc 6 1
data = list(zip(*[map(int, line.split()[1:]) for line in inputs.strip().split("\n")]))
answer = 1
for t, d in data:
    answer *= t - 1 - 2*math.floor((t - math.sqrt(t**2 - 4*d))/2)

Execution time: 0.00095 seconds
Correct!


In [15]:
%%aoc 6 2
t, d = map(int, ["".join(line.split()[1:]) for line in inputs.strip().split("\n")])
answer = t - 1 - 2*math.floor((t - math.sqrt(t**2 - 4*d))/2)

Execution time: 0.00081 seconds
Correct!


# Day 7

In [16]:
%%aoc 7 1

types = [(5, ), (1, 4), (2, 3), (1, 1, 3), (1, 2, 2), (1, 1, 1, 2), (1, 1, 1, 1, 1)]
cards = ["A", "K", "Q", "J", "T", "9", "8", "7", "6", "5", "4", "3", "2"]

def hand_type(hand):
    hand_type = types.index(tuple(sorted(len(list(x[1])) for x in itertools.groupby(sorted(hand)))))
    first_card = [cards.index(card) for card in hand]
    return (hand_type, first_card)

lines = [line.split(" ") for line in inputs.strip().split("\n")]

hands = sorted(lines, key=lambda x: hand_type(x[0]), reverse=True)

answer = 0
for i, (_, bid ) in enumerate(hands, 1):
    answer += i * int(bid)

Execution time: 0.0074 seconds
Correct!


In [17]:
%%aoc 7 2

types = [(5, ), (1, 4), (2, 3), (1, 1, 3), (1, 2, 2), (1, 1, 1, 2), (1, 1, 1, 1, 1)]
cards = ["A", "K", "Q", "T", "9", "8", "7", "6", "5", "4", "3", "2", "J"]

def hand_type(hand):
    counts = tuple(sorted(len(list(x[1])) for x in itertools.groupby(sorted(hand.replace("J", "")))))
    for i, t in enumerate(types):
        if len(counts) <= len(t) and counts <= t:
            hand_type = i
            break
    
    first_card = [cards.index(card) for card in hand]
    return (hand_type, first_card)


lines = [line.split(" ") for line in inputs.strip().split("\n")]

hands = sorted(lines, key=lambda x: hand_type(x[0]), reverse=True)

answer = 0
for i, (_, bid ) in enumerate(hands, 1):
    answer += i * int(bid)

Execution time: 0.0084 seconds
Correct!


# Day 9

In [18]:
%%aoc 9 1

def predict(seq):
    diffs = [y - x for x, y in zip(seq[:-1], seq[1:])]
    if set(diffs) == {0}:
        return seq[-1]

    return seq[-1] + predict(diffs)

seqs = [[int(x) for x in line.split()] for line in inputs.strip().split("\n")]

answer = sum(predict(seq) for seq in seqs)

Execution time: 0.0074 seconds
Correct!


In [19]:
%%aoc 9 2

def predict(seq):
    diffs = [y - x for x, y in zip(seq[:-1], seq[1:])]
    if set(diffs) == {0}:
        return seq[0]

    return seq[0] - predict(diffs)

seqs = [[int(x) for x in line.split()] for line in inputs.strip().split("\n")]

answer = sum(predict(seq) for seq in seqs)

Execution time: 0.0075 seconds
Correct!
